In [3]:
compress = '100%'

# = = = = = = = = = = = = = = =

from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import requests

from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import PIL
from io import BytesIO
from openpyxl.drawing.image import Image

# = = = = = = = = = = = = = = =

wb = load_workbook('./crawler.xlsx')
ws = wb.active

for column in range(1, ws.max_column + 1):
    if ws.cell(1, column).value == 'Json_Src':
        column_src = column
    elif ws.cell(1, column).value == 'Picture':
        column_pic = column

print(f'总数量：{ws.max_row - 1}')

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(2, ws.max_row + 1):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global ws

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =
        
        src = json.loads(ws.cell(a, column_src).value).get('0', '')

        # = = = = = = = = = = = = = = =

        if src == '':
            b = 0
            crawler_status = 'ok'
        else:
            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.get(src,
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200:
                            break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                image = PIL.Image.open(BytesIO(resp.content)).convert('RGB')

                # = = = = = = = = = = = = = = =

                if image.width >= image.height and image.width > 500:
                    image = image.resize((500, int(image.height / (image.width / 500))), PIL.Image.Resampling.LANCZOS)
                elif image.width < image.height and image.height > 500:
                    image = image.resize((int(image.width / (image.height / 500)), 500), PIL.Image.Resampling.LANCZOS)

                # = = = = = = = = = = = = = = =

                buffer = BytesIO()
                if compress == '100%':
                    image.save(buffer, format="PNG")
                else:
                    image.save(buffer, format="JPEG", quality=int(compress.removesuffix('%')))
                buffer.seek(0)
                
                pic = Image(buffer)

                # = = = = = = = = = = = = = = =

                ws.add_image(pic, f'{get_column_letter(column_pic)}{a}')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                pass

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b}] - {src}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(30):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
wb.save(f'''./pic-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''')
print('Done ~')

[状态：ok，尝试次数：10] - https://www.rockauto.com/info/52/39649.jpg
[剩余数量：1000] - [当前时间：18:13:28]

[状态：ok，尝试次数：7] - https://www.rockauto.com/info/52/168342.jpg
[剩余数量：999] - [当前时间：18:13:28]

[状态：ok，尝试次数：5] - https://www.rockauto.com/info/52/39631.jpg
[剩余数量：998] - [当前时间：18:13:28]

[状态：ok，尝试次数：13] - https://www.rockauto.com/info/52/39622.jpg
[剩余数量：997] - [当前时间：18:13:28]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/52/168339.jpg
[剩余数量：996] - [当前时间：18:13:29]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/52/83406.jpg
[剩余数量：995] - [当前时间：18:13:29]

[状态：ok，尝试次数：2] - https://www.rockauto.com/info/52/39646.jpg
[剩余数量：994] - [当前时间：18:13:29]

[状态：ok，尝试次数：17] - https://www.rockauto.com/info/52/39626.jpg
[剩余数量：993] - [当前时间：18:13:29]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/52/83355.jpg
[剩余数量：992] - [当前时间：18:13:29]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/52/39647.jpg
[剩余数量：991] - [当前时间：18:13:29]

[状态：ok，尝试次数：1] - https://www.rockauto.com/info/52/39661.jpg
[剩余数量：990] - [当前时间：18:13:29]

[状态：